## Stock screener by querying from finviz.com

## OVERVIEW
This tool will populate top stocks to pick by querying information from www.finviz.com

## Methodology

### Fundamental and Descriptive
Query all the stocks with following criteria:

URL: https://finviz.com/screener.ashx?v=111&ft=2 (to check fundamentals below)

1. EPS growth next 5 years > 10%
2. Return on Equity > 10%
3. EPS growth past 5 years > 10%
4. Sales growth past 5 years > 10%, and
5. Debt/Equity < 0.5

URL: https://finviz.com/screener.ashx?v=111&f=fa_debteq_u0.5,fa_eps5years_o10,fa_estltgrowth_o10,fa_roe_o10,fa_sales5years_o10&ft=2
(after filtering above 5 criteria)

Also, put filters on stock descriptions:

URL: https://finviz.com/screener.ashx?v=111 (to check descriptive information below)

1. Country: USA
2. Option/Short: optionable

URL: https://finviz.com/screener.ashx?v=111&f=geo_usa,sh_opt_option (after filtering above)

Choose "custom" columns: Ticker, Sector, Industry, Market Cap, P/E, EPS, EPS next 5Y, ROE, Price, Volume

Read all the pages after making choices above (dataframe)
skip tickers with missing data for any of the 5 fundamental criteria

### Method

Calculate the following:

Rule_1_growth = min(ROE, EPS next 5Y)
Rule_1_PE = min(P/E, 2 x Rule_1_growth x 100)
Future_EPS = EPS*(1 + Rule_1_PE)^10
Future_Price = Future_EPS x Rule_1_PE
Future_Price_Est = Future_Price/4
MoS_Price = 0.5 x Future_Price_Est

list of good stocks:
Good_Stocks = if MoS_Price < Price

Sort list of Good_stocks by MoS_Price/Price

### Additional

#TODO: When to sell

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline 

In [2]:
url = "https://finviz.com/screener.ashx?v=111&f=fa_debteq_u0.5,fa_eps5years_o10,fa_estltgrowth_o10,fa_roe_o10,fa_sales5years_o10&ft=2"

In [3]:
df = pd.read_html(url)

HTTPError: HTTP Error 403: Forbidden

In [8]:
url = "https://finviz.com/screener.ashx?v=111&ft=2"

In [9]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen

pd.set_option('display.max_colwidth', 25)

# Set up scraper
url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

In [40]:
url

'http://finviz.com/quote.ashx?t=tsla'

In [5]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
from urllib.request import Request, urlopen

pd.set_option('display.max_colwidth', 25)

# Input
symbol = input('Enter a ticker: ')
print ('Getting data for ' + symbol + '...\n')

# Set up scraper
url = ("http://finviz.com/quote.ashx?t=" + symbol.lower())
req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
html = soup(webpage, "html.parser")

def get_fundamentals():
    try:
        # Find fundamentals table
        fundamentals = pd.read_html(str(html), attrs = {'class': 'snapshot-table2'})[0]
        
        # Clean up fundamentals dataframe
        fundamentals.columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
        colOne = []
        colLength = len(fundamentals)
        for k in np.arange(0, colLength, 2):
            colOne.append(fundamentals[f'{k}'])
        attrs = pd.concat(colOne, ignore_index=True)
    
        colTwo = []
        colLength = len(fundamentals)
        for k in np.arange(1, colLength, 2):
            colTwo.append(fundamentals[f'{k}'])
        vals = pd.concat(colTwo, ignore_index=True)
        
        fundamentals = pd.DataFrame()
        fundamentals['Attributes'] = attrs
        fundamentals['Values'] = vals
        fundamentals = fundamentals.set_index('Attributes')
        return fundamentals

    except Exception as e:
        return e
    
def get_news():
    try:
        # Find news table
        news = pd.read_html(str(html), attrs = {'class': 'fullview-news-outer'})[0]
        links = []
        for a in html.find_all('a', class_="tab-link-news"):
            links.append(a['href'])
        
        # Clean up news dataframe
        news.columns = ['Date', 'News Headline']
        news['Article Link'] = links
        news = news.set_index('Date')
        return news

    except Exception as e:
        return e

def get_insider():
    try:
        # Find insider table
        insider = pd.read_html(str(html), attrs = {'class': 'body-table'})[0]
        
        # Clean up insider dataframe
        insider = insider.iloc[1:]
        insider.columns = ['Trader', 'Relationship', 'Date', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']
        insider = insider[['Date', 'Trader', 'Relationship', 'Transaction', 'Cost', '# Shares', 'Value ($)', '# Shares Total', 'SEC Form 4']]
        insider = insider.set_index('Date')
        return insider

    except Exception as e:
        return e

print ('Fundamental Ratios: ')
print(get_fundamentals())

print ('\nRecent News: ')
print(get_news())

print ('\nRecent Insider Trades: ')
print(get_insider())

Enter a ticker: TSLA
Getting data for TSLA...

Fundamental Ratios: 
                 Values
Attributes             
Index           S&P 500
Market Cap      755.77B
Income          690.00M
Sales            31.54B
Book/sh           23.37
...                 ...
ATR               34.32
Volatility  3.71% 3.93%
Prev Close       787.38
Price            782.22
Change           -0.65%

[72 rows x 1 columns]

Recent News: 
                              News Headline              Article Link
Date                                                                 
Feb-19-21 02:48PM  Bitcoin Hits $1 Trill...  https://finance.yahoo...
02:44PM            Cantor CEO Lutnick Li...  https://finance.yahoo...
02:37PM            Big Tech is swallowin...  https://www.marketwat...
02:27PM            Bill Gates on sustain...  https://finance.yahoo...
01:51PM            Morgan Stanley sees G...  https://finance.yahoo...
...                                     ...                       ...
12:02PM            Dow

In [39]:
pd.read_html(str(html), attrs = {'class': 'body-table'})

[                    0                         1       2                3  \
 0     Insider Trading              Relationship    Date      Transaction   
 1    Baglino Andrew D  SVP Powertrain and En...  Feb 10  Option Exercise   
 2    Guillen Jerome M     President, Automotive  Feb 10  Option Exercise   
 3    Guillen Jerome M     President, Automotive  Feb 10             Sale   
 4    Baglino Andrew D  SVP Powertrain and En...  Feb 10             Sale   
 ..                ...                       ...     ...              ...   
 96   Kirkhorn Zachary   Chief Financial Officer  Jun 05  Option Exercise   
 97     Taneja Vaibhav  Chief Accounting Officer  Jun 05  Option Exercise   
 98   Guillen Jerome M     President, Automotive  Jun 05  Option Exercise   
 99   Guillen Jerome M     President, Automotive  Jun 01  Option Exercise   
 100       Musk Kimbal                  Director  Jun 01  Option Exercise   
 
           4        5          6              7                8  
 0     

In [43]:
type(insider.to_dict(orient="records"))

list

In [44]:
fundamentals = get_fundamentals()

In [52]:
news = get_news()

In [58]:
news.to_dict(orient="records")

[{'News Headline': 'Bitcoin Hits $1 Trillion Value as Crypto Jump Tops Other Assets Bloomberg',
  'Article Link': 'https://finance.yahoo.com/news/bitcoin-nears-1-trillion-value-055731771.html'},
 {'News Headline': 'Cantor CEO Lutnick Likens Tesla, Bitcoin Surges to GameStop Saga Bloomberg',
  'Article Link': 'https://finance.yahoo.com/news/cantor-ceo-lutnick-likens-tesla-194446070.html'},
 {'News Headline': 'Big Tech is swallowing the rest of Silicon Valley MarketWatch',
  'Article Link': 'https://www.marketwatch.com/story/big-tech-is-swallowing-the-rest-of-silicon-valley-11613763453?siteid=yhoof2'},
 {'News Headline': "Bill Gates on sustainable investing: 'There's probably some Teslas out there' Yahoo Finance",
  'Article Link': 'https://finance.yahoo.com/news/bill-gates-investments-clean-energy-tesla-192707286.html'},
 {'News Headline': 'Morgan Stanley sees GM SPACtopus taking on EV market Yahoo Finance',
  'Article Link': 'https://finance.yahoo.com/news/morgan-stanley-sees-gm-spa-ct

In [51]:
insider.to_dict(orient="records")

[{'Trader': 'Baglino Andrew D',
  'Relationship': 'SVP Powertrain and Energy Eng.',
  'Transaction': 'Option Exercise',
  'Cost': '51.64',
  '# Shares': '4000',
  'Value ($)': '206560',
  '# Shares Total': '23226',
  'SEC Form 4': 'Feb 12 09:10 PM'},
 {'Trader': 'Guillen Jerome M',
  'Relationship': 'President, Automotive',
  'Transaction': 'Option Exercise',
  'Cost': '55.32',
  '# Shares': '10000',
  'Value ($)': '553200',
  '# Shares Total': '59959',
  'SEC Form 4': 'Feb 12 07:55 PM'},
 {'Trader': 'Guillen Jerome M',
  'Relationship': 'President, Automotive',
  'Transaction': 'Sale',
  'Cost': '817.58',
  '# Shares': '10000',
  'Value ($)': '8175763',
  '# Shares Total': '49959',
  'SEC Form 4': 'Feb 12 07:55 PM'},
 {'Trader': 'Baglino Andrew D',
  'Relationship': 'SVP Powertrain and Energy Eng.',
  'Transaction': 'Sale',
  'Cost': '807.89',
  '# Shares': '4000',
  'Value ($)': '3231540',
  '# Shares Total': '19226',
  'SEC Form 4': 'Feb 12 09:10 PM'},
 {'Trader': 'Musk Kimbal',
  '

In [49]:
fundamentals.to_dict(orient="dict")

{'Values': {'Index': 'S&P 500',
  'Market Cap': '755.77B',
  'Income': '690.00M',
  'Sales': '31.54B',
  'Book/sh': '23.37',
  'Cash/sh': '20.19',
  'Dividend': '-',
  'Dividend %': '-',
  'Employees': '70757',
  'Optionable': 'Yes',
  'Shortable': 'Yes',
  'Recom': '2.90',
  'P/E': '1265.88',
  'Forward P/E': '145.60',
  'PEG': '35.46',
  'P/S': '23.97',
  'P/B': '33.69',
  'P/C': '38.99',
  'P/FCF': '128.80',
  'Quick Ratio': '1.60',
  'Current Ratio': '1.90',
  'Debt/Eq': '0.53',
  'LT Debt/Eq': '0.43',
  'SMA20': '-6.66%',
  'EPS (ttm)': '0.62',
  'EPS next Y': '29.84%',
  'EPS next Q': '0.76',
  'EPS this Y': '165.00%',
  'EPS next 5Y': '35.70%',
  'EPS past 5Y': '19.70%',
  'Sales past 5Y': '50.80%',
  'Sales Q/Q': '-56.30%',
  'EPS Q/Q': '123.80%',
  'Earnings': 'Jan 27 AMC',
  'SMA50': '2.16%',
  'Insider Own': '0.10%',
  'Insider Trans': '-41.23%',
  'Inst Own': '43.00%',
  'Inst Trans': '2.69%',
  'ROA': '1.60%',
  'ROE': '4.80%',
  'ROI': '5.00%',
  'Gross Margin': '21.00%',